In [16]:
!pip install grad-cam

In [17]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch    
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image

from models.retinanet import my_retinanet_resnet50_fpn





COLORS = np.random.uniform(0, 255, size=(80, 3))


image_url = "http://farm1.staticflickr.com/180/460801098_5c15f591db_z.jpg"
img = np.array(Image.open(requests.get(image_url, stream=True).raw))
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255
transform = transforms.ToTensor()
tensor = transform(img).unsqueeze(0)

# customized model
model = my_retinanet_resnet50_fpn()
model.eval()
model.cpu()

s3 = model.backbone.body.layer2[3].conv3
s4 = model.backbone.body.layer3[5].conv3
s5 = model.backbone.body.layer4[2].conv3
target_layers = [s3, s4, s5]

results = model(tensor)
print(results)

return_layers: {'layer2': '0', 'layer3': '1', 'layer4': '2'}
extra_blocks: LastLevelP6P7(
  (p6): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
)
[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'scores': tensor([], grad_fn=<IndexBackward0>), 'labels': tensor([], dtype=torch.int64)}]


In [18]:
cam = EigenCAM(model, target_layers)
grayscale_cam = cam(tensor)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
Image.fromarray(cam_image)

AttributeError: 'list' object has no attribute 'cpu'